In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F

a:\python\312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data.csv')

df['comment'] = df['comment'].str.replace('\n', '', regex=False)  # Удаляем символы новой строки

# Разделение данных на обучающую и тестовую выборки
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Токенизация
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

class ToxicCommentsDataset(Dataset):
    def __init__(self, comments, labels):
        self.comments = comments
        self.labels = labels
        
    def __len__(self):
        return len(self.comments)
    
    def __getitem__(self, idx):
        comment = self.comments[idx]
        label = self.labels[idx]
        
        # Токенизация
        inputs = tokenizer(comment, padding='max_length', truncation=True, return_tensors="pt", max_length=128)
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Создание датасетов
train_dataset = ToxicCommentsDataset(train_df['comment'].tolist(), train_df['toxic'].tolist())
test_dataset = ToxicCommentsDataset(test_df['comment'].tolist(), test_df['toxic'].tolist())

# Загрузка данных
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, pin_memory=True)

In [ ]:
class PowerfulBinaryTextClassifier(nn.Module):
    def __init__(self, model_name, lstm_hidden_size=256, num_layers=3, dropout_rate=0.2):
        super(PowerfulBinaryTextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        
        # Добавляем несколько LSTM слоев с большим размером скрытого состояния
        self.lstm = nn.LSTM(input_size=self.bert.config.hidden_size, 
                            hidden_size=lstm_hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True, 
                            bidirectional=True, 
                            dropout=dropout_rate if num_layers > 1 else 0)
        
        # Полносвязный блок с увеличенным количеством нейронов и слоев Dropout
        self.fc = nn.Sequential(
            nn.Linear(lstm_hidden_size * 2, 2),  # полносвязный слой
            nn.Sigmoid()
        )
        
        self.tokenizer = BertTokenizer.from_pretrained(model_name)  # Инициализация токенизатора
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_outputs = outputs.last_hidden_state  # (batch_size, sequence_length, hidden_size)
        
        # Применяем LSTM
        lstm_out, _ = self.lstm(bert_outputs)  # (batch_size, sequence_length, lstm_hidden_size * 2)
        
        # Берем выход последнего временного шага для классификации
        last_time_step = lstm_out[:, -1, :]  # (batch_size, lstm_hidden_size * 2)
        
        logits = self.fc(last_time_step)  # Применяем полносвязный блок
        return logits  # Возвращаем логиты для двух классов

    def predict(self, text):
        self.to(self.device)  # Переносим модель на выбранное устройство
        
        # Токенизация текста
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
        input_ids = inputs['input_ids'].to(self.device)  # Переносим на устройство
        attention_mask = inputs['attention_mask'].to(self.device)  # Переносим на устройство
        
        # Получение предсказания
        self.eval()  # Переключаем модель в режим оценки
        with torch.no_grad():
            preds = self(input_ids, attention_mask)  # Получаем логиты
        
        # Возвращаем индекс класса с наибольшей вероятностью
        return torch.argmax(preds, dim=1).item()  # Возвращаем индекс класса

    def load_weights(self, filepath):
        # Загрузка весов модели
        self.load_state_dict(torch.load(filepath, map_location=self.device))

# Пример инициализации модели
model_name = "DeepPavlov/rubert-base-cased"
model = PowerfulBinaryTextClassifier(model_name)

# Пример использования
text = "Привет, это хороший день."
predicted_class_index = model.predict(text)
print(f'Результат прогноза текста "{text}": {predicted_class_index}')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Результат прогноза текста "Привет, это хороший день.": 0


In [4]:
import torch.optim as optim
from tqdm import tqdm  # Для отображения прогресса обучения
from sklearn.metrics import accuracy_score, f1_score  # Импортируем метрики

# Установка параметров обучения
num_epochs = 3
learning_rate = 2e-5

# Инициализация модели, функции потерь и оптимизатора
model = PowerfulBinaryTextClassifier(model_name).to(device)
criterion = nn.CrossEntropyLoss()  # Функция потерь для многоклассовой классификации
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Функция для оценки модели
def evaluate_model(model, test_loader):
    model.eval()  # Переключаем модель в режим оценки
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Отключаем подсчет градиентов для оценки
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Прямой проход
            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()  # Получаем предсказания

            # Сохраняем предсказания и истинные метки
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Вычисляем метрики
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f'Точность на тестовых данных: {accuracy:.4f}, F1-мера: {f1:.4f}')
    
# Функция для обучения модели
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Переключаем модель в режим обучения
        total_loss = 0
        all_labels = []  # Для хранения всех меток
        all_preds = []   # Для хранения всех предсказаний
        
        for batch in tqdm(train_loader, desc=f'Эпоха {epoch+1}/{num_epochs}', leave=False):
            optimizer.zero_grad()  # Обнуляем градиенты

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Прямой проход
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels.long())  # Применяем функцию потерь
            total_loss += loss.item()

            # Обратный проход и обновление параметров
            loss.backward()
            optimizer.step()

            # Получаем предсказания
            preds = torch.argmax(outputs, dim=1).cpu().numpy()  # Предсказания
            all_preds.extend(preds)  # Добавляем предсказания в общий список
            all_labels.extend(labels.cpu().numpy())  # Добавляем истинные метки в общий список
        
        print(f'Эпоха {epoch+1}', end=' ')
        evaluate_model(model, test_loader)

# Запуск процесса обучения
train_model(model, train_loader, criterion, optimizer, num_epochs)

# Сохранение модели после обучения
torch.save(model.state_dict(), 'model.pth')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Эпоха 1 Точность на тестовых данных: 0.7787, F1-мера: 0.7438


Эпоха 2 Точность на тестовых данных: 0.9136, F1-мера: 0.9144


Эпоха 3 Точность на тестовых данных: 0.9126, F1-мера: 0.9132


In [5]:
# Пример использования 
comments = [
    # Токсичные комментарии (Класс: 1)
    "Ты в своем уме? Как можно быть таким тупым и неадекватным?",
    "Этот бред просто поражает, как можно так не понимать простые вещи?",
    "Как же ты надоел своими безумными идеями, убирайся к черту!",
    "Сколько можно тратить время на твои идиотские комментарии?",
    "Кто тебе дал право говорить такие глупости? Умоляю, замолчи!",
    "Ты как всегда на высоте — высоте своего безумия.",
    "Ужас, просто ужас. Надеюсь, ты хоть сам понимаешь, как ты выглядишь.",
    "Почему ты не можешь оставить свои бредовые мысли при себе?",
    "Твои слова — это просто смех и позор, не позорься больше!",
    "Слушай, может, тебе стоит просто уйти и больше не возвращаться?",

    # Нетоксичные комментарии (Класс: 0)
    "Недавно посмотрел интересный фильм — действительно затянуло!",
    "Каждый день пробую новые рецепты — это так увлекательно.",
    "Недавно начал читать новую книгу, и она просто захватывающая.",
    "Погода сегодня прекрасная, надеюсь, выйду на прогулку!",
    "Вчера попробовал новый кофе — он оказался невероятно вкусным.",
    "В выходные собираюсь съездить на природу, очень жду этого!",
    "Занимаюсь спортом, и это приносит много радости в жизнь.",
    "Посмотрел новый сериал, и он оказался очень увлекательным.",
    "В последнее время увлекся рисованием — это очень расслабляет.",
    "На выходных встретился с друзьями — было действительно весело!"
]

for text in comments:
    predicted_class_index = model.predict(text) # прогноз класса текста
    print(f'Текст: "{text}" Класс: {predicted_class_index}')

Текст: "Ты в своем уме? Как можно быть таким тупым и неадекватным?" Класс: 1
Текст: "Этот бред просто поражает, как можно так не понимать простые вещи?" Класс: 0
Текст: "Как же ты надоел своими безумными идеями, убирайся к черту!" Класс: 1
Текст: "Сколько можно тратить время на твои идиотские комментарии?" Класс: 1
Текст: "Кто тебе дал право говорить такие глупости? Умоляю, замолчи!" Класс: 1
Текст: "Ты как всегда на высоте — высоте своего безумия." Класс: 1
Текст: "Ужас, просто ужас. Надеюсь, ты хоть сам понимаешь, как ты выглядишь." Класс: 1
Текст: "Почему ты не можешь оставить свои бредовые мысли при себе?" Класс: 1
Текст: "Твои слова — это просто смех и позор, не позорься больше!" Класс: 1
Текст: "Слушай, может, тебе стоит просто уйти и больше не возвращаться?" Класс: 1
Текст: "Недавно посмотрел интересный фильм — действительно затянуло!" Класс: 0
Текст: "Каждый день пробую новые рецепты — это так увлекательно." Класс: 0
Текст: "Недавно начал читать новую книгу, и она просто захват